In [0]:
file_path = "dbfs:/FileStore/tables/Vipul/Sample___Superstore.csv"
df = spark.read.option("header", True).option("inferSchema", True).csv(file_path)


In [0]:
for col_name in df.columns:
    df = df.withColumnRenamed(col_name, col_name.strip().lower().replace(" ", "_"))


In [0]:
from pyspark.sql.functions import expr

df = (
    df
    .withColumn("sales_clean",    expr("try_cast(sales    AS double)"))
    .withColumn("profit_clean",   expr("try_cast(profit   AS double)"))
    .withColumn("discount_clean", expr("try_cast(discount AS double)"))
)

In [0]:
# STEP 4 ─ Feature‑engineering: profit_margin, discount_category, order_month ------------
from pyspark.sql.functions import when, round, to_date, year, month, concat_ws, col

# 4‑a  profit_margin  (NULL if sales_clean is 0 or NULL)
df = df.withColumn(
    "profit_margin",
    when(col("sales_clean") == 0, None)
    .otherwise(round(col("profit_clean") / col("sales_clean"), 2))
)

# 4‑b  discount_category  (based on discount_clean)
df = df.withColumn(
    "discount_category",
    when(col("discount_clean") >= 0.30, "High")
    .when(col("discount_clean") >= 0.10, "Medium")
    .otherwise("Low")
)

# 4‑c  order_month  (YYYY‑M) for easy partitioning / trend plots
df = (
    df
    .withColumn("order_date", to_date("order_date", "MM/dd/yyyy"))
    .withColumn("order_month", concat_ws("-", year("order_date"), month("order_date")))
)

In [0]:
df = df.drop("sales", "profit", "discount")


In [0]:
df.write.format("delta").mode("overwrite").save("/dbfs/FileStore/delta/clean_superstore")


In [0]:
df_delta = spark.read.format("delta").load("/dbfs/FileStore/delta/clean_superstore")
df_delta.show(5, truncate=False)

In [0]:
df_delta.createOrReplaceTempView("delta_clean_superstore")

In [0]:
%sql
SELECT discount_category, COUNT(*) AS total_orders
FROM delta_clean_superstore
GROUP BY discount_category
ORDER BY total_orders DESC


Databricks visualization. Run in Databricks to view.

## Trend analysis

In [0]:
%sql
SELECT order_month, ROUND(SUM(sales_clean), 2) AS total_sales
FROM delta_clean_superstore
GROUP BY order_month
ORDER BY order_month


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT order_month, COUNT(*) AS total_orders
FROM delta_clean_superstore
GROUP BY order_month
ORDER BY order_month


Databricks visualization. Run in Databricks to view.

In [0]:
# Re-load cleaned DataFrame from CSV if needed
# Or use df_delta from earlier notebook

# IMPORTANT: Use this path (without /dbfs prefix)
df_delta.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save("dbfs:/FileStore/delta/clean_superstore")


In [0]:
from delta.tables import DeltaTable

DeltaTable.forPath(spark, "dbfs:/FileStore/delta/clean_superstore")  # ✅ No error = it’s a Delta table


In [0]:
%sql
OPTIMIZE delta.`dbfs:/FileStore/delta/clean_superstore`
